## NE_Reval_2021_adhoc

E. Quinn 5/20/2021

Process NE Reval 2021 valuations data



## Import standard python datascience packages

In [ ]:
import sys
import math
import re
import copy as cp
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
#plt.switch_backend('WebAgg')
import seaborn as sns
import pickle
%matplotlib inline

In [ ]:
from datetime import datetime, timedelta, date
from datascience import *
import uuid
import random

### Show the directory we are running in

In [ ]:
!pwd

### Load NE Reval data scraped from website

In [ ]:
with open('../NE_Reval_dictionary_5_18_2021.pkl', 'rb') as handle:
    itm = pickle.load(handle)

In [ ]:
R2020 = 23.43
R2021 = 21.91

In [ ]:
parcels = {}

for pkl in itm.keys():
    dct = itm[pkl]
    try:
        parcelID = dct['ParcelID_ParcelID']['ParcelID']
        state = dct['ParcelID_ParcelID']['State']
        if (state == '01 - Sngle Fam'):
            if parcelID not in parcels.keys():
                parcels[parcelID] = {}
                parcels[parcelID]['State'] = state
            parcels[parcelID]['Address'] = itm[pkl]['LocationOwner_Location']['Location']
            parcels[parcelID]['R2021'] = R2021
            parcels[parcelID]['R2020'] = R2020
            for i in dct['PriorInformation_GridView2'].keys():
                if (dct['PriorInformation_GridView2'][i]['FY'] == '2021'):
                    parcels[parcelID]['V2021'] = float(dct['PriorInformation_GridView2'][i]['Total_value'])
                if (dct['PriorInformation_GridView2'][i]['FY'] == '2020'):
                    parcels[parcelID]['V2020'] = float(dct['PriorInformation_GridView2'][i]['Total_value'])
            parcels[parcelID]['deltaR'] = R2021 - R2020
            parcels[parcelID]['deltaV'] = (parcels[parcelID]['V2021'] - parcels[parcelID]['V2020'])/1000.0
            parcels[parcelID]['rate_effect'] = parcels[parcelID]['V2020']*parcels[parcelID]['deltaR']/1000.0
            parcels[parcelID]['valuation_effect'] = parcels[parcelID]['R2020']*parcels[parcelID]['deltaV']
            parcels[parcelID]['interaction'] = parcels[parcelID]['deltaV']*parcels[parcelID]['deltaR']
            parcels[parcelID]['gross_tax2020'] = round(R2020*parcels[parcelID]['V2020']/1000.0,2)
            parcels[parcelID]['gross_tax2021'] = round(R2021*parcels[parcelID]['V2021']/1000.0,2)
            parcels[parcelID]['gross_tax_delta'] = round((parcels[parcelID]['gross_tax2021'] -\
                                        parcels[parcelID]['gross_tax2020']),2)
            parcels[parcelID]['gross_tax_pct_chg'] = round(100.0*(parcels[parcelID]['gross_tax_delta']/\
                                        parcels[parcelID]['gross_tax2020']),2)
        
    except KeyError:
        print('KeyError:',pkl)
        
                
len(parcels)

### Display one dictionary entry

In [ ]:
valuation2021 = []
valuation2020 = []
valuation_pct_chg = []
gross_tax_pct_chg = []
rate_effect = []
valuation_effect = []
interaction = []
gross_tax_delta = []

for parcel in parcels.keys():
    valuation2021.append(parcels[parcel]['V2021'])
    valuation2020.append(parcels[parcel]['V2020'])
    rate_effect.append(parcels[parcel]['rate_effect'])
    valuation_effect.append(parcels[parcel]['valuation_effect'])
    interaction.append(parcels[parcel]['interaction'])
    gross_tax_delta.append(parcels[parcel]['gross_tax_delta'])
    gross_tax_pct_chg.append(parcels[parcel]['gross_tax_pct_chg'])
    valuation_pct_chg.append(round(100.0*(parcels[parcel]['V2021']-parcels[parcel]['V2020'])/\
                                   parcels[parcel]['V2020'],2))
    
data = {'2021 Valuation':valuation2021,'2020 Valuation':valuation2020,'Valuation Percent Change':valuation_pct_chg,\
           'Gross Tax Pct Change':gross_tax_pct_chg,'Gross Tax Delta':gross_tax_delta,'Rate Effect':rate_effect,\
        'Valuation Effect':valuation_effect,'Interaction':interaction}

df = pd.DataFrame (data, columns = ['2021 Valuation','2020 Valuation','Valuation Percent Change',\
            'Gross Tax Pct Change','Gross Tax Delta','Rate Effect','Valuation Effect','Interaction'])

df.head()

### Quantiles for various quantities of interest

In [ ]:
q = [.01,0.1,0.25,0.5,0.75,0.9,.99]

print('Single family home 2021 valuation quantiles',np.quantile(valuations,q))
print('Single family home 2021 valuation percent change',np.quantile(valuation_pct_chg,q))
print('Single family home 2021 gross tax percent change',np.quantile(gross_tax_pct_chg,q))


In [ ]:
fig, ax = plt.subplots()
sns.histplot(data=df, x="Valuation Percent Change", ax=ax)
ax.set_xlim(-10,40)
ax.set_title('Single Family Homes: Percent Change in Valuation',fontsize = 22)
ax.set_xlabel('Percent Change',fontsize = 20, )
ax.set_ylabel('Number of Homes',fontsize = 20, )
fig.set_size_inches(12, 8)

In [ ]:
fig, ax = plt.subplots()
sns.histplot(data=df, x="Gross Tax Pct Change", ax=ax)
ax.set_xlim(-10,40)
ax.set_title('Single Family Homes: Gross Tax Percent Change',fontsize = 22)
ax.set_xlabel('Percent Change',fontsize = 20, )
ax.set_ylabel('Number of Homes',fontsize = 20, )
fig.set_size_inches(12, 8)

### Variance components for gross tax delta

In [ ]:
print('gross tax delta variance:',round(np.var(np.array(gross_tax_delta))))
print('rate effect variance:',round(np.var(np.array(rate_effect))))
print('valuation effect variance:',round(np.var(np.array(valuation_effect))))
print('interaction variance:',round(np.var(np.array(interaction))))
print('rate-valuation covariance:',round(np.cov(np.array(rate_effect),np.array(valuation_effect))[0][1]))
print('rate-interaction covariance:',round(np.cov(np.array(rate_effect),np.array(interaction))[0][1]))
print('valuation-interaction covariance:',round(np.cov(np.array(valuation_effect),np.array(interaction))[0][1]))

In [ ]:
with open('../tax_bill_changes.html', 'w') as htmlout:
    
    htmlout.write("<html><head><body>\n")
    htmlout.write("<h2>Effect of Tax Rate and Valuation Changes on Tax Bill</h2>\n")
    htmlout.write("<table border='1' width='100%'>\n")
    htmlout.write("<tr><td>Parcel</td><td>Address</td><td>2021 Valuation</td><td>2020 Valuation</td><td>" +\
                  "2021 Gross Tax</td><td>2020 Gross Tax</td><td>Tax Delta</td>" +\
                  "<td>Effect of Rate Change</td><td>Effect of Valuation Change</td>" +\
                  "<td>Interaction Effect</td></tr>\n")
    for parcel in sorted(parcels.keys()):
        s = '<tr align="right"><td>' + parcel + '</td><td>' + parcels[parcel]['Address'] + '</td><td>' +\
            str(parcels[parcel]['V2021']) + '</td><td>' + str(parcels[parcel]['V2020']) +\
            '</td><td>' + str(parcels[parcel]['gross_tax2021']) +\
            '</td><td>' + str(parcels[parcel]['gross_tax2020']) + '</td><td>' +\
            str(parcels[parcel]['gross_tax_delta']) +'</td><td>' + str(round(parcels[parcel]['rate_effect'],2)) +\
            '</td><td>' + str(round(parcels[parcel]['valuation_effect'],2)) + '</td><td>' +\
            str(round(parcels[parcel]['interaction'],2)) +\
            '</td></tr>\n'
        htmlout.write(s)

    htmlout.write("</table></body></html")